# <u>Sea Level Rise Threat Index Tool </u>

#### Meg Manning -- *Duke University, Fall 2024*

► **Description:** this script can be used to identify an area in the coastal plain of North Carolina and assess it's risk to sea level rise. The overall threat index is calculated through combining the ranked exposure of inundation under 1ft, 2ft, and 3ft of sea level rise, flood prone areas, storm surge under category 1-5 hurricanes, soil erodibility, soil drainage, and areas of low slope for a given area of interest. Methods for forecasting sea level rise inundation were taken from the National Oceanic and Atmospheric detailed methology for sea level rise inundation (NOAA, 2017) and the overall threat index was modelled after the National Fish and Wildlife Service's Coastal Resilience and Siting Tool (CREST) (Dobson et al., 2019).

### 1. Digital Elevation Model from LiDAR 

*Download LiDAR point data and process into a 1m DEM.*

#### Since there is no API for a direct download of LiDAR point data, in order to process the rest of this script, data must be downloaded following these steps *prior* to running the rest of this script. 

<u> Step 1: Download LiDAR Data </u> 

Click the link below to open the Digital Coast Topobathy LiDAR Web Interface: https://coast.noaa.gov/dataviewer/#/lidar/search/ 

<u>Step 1.1:</u> Zoom in to a scale of at least 1000ft, if not 500ft, and draw a box around the area of interst

<u>Step 1.2:</u> In the panel on the right, select the most recent *point* data that corresponds to the area of interest

<u>Step 1.3:</u> Click "Add to Cart" 

<u>Step 1.4:</u> In your cart, set the following parameters for your point data:
* Projection = UTM
* Zone = Zone 18 Range 078W-072W
* Horizontal Datum = NAD83
* Horizontal Units = Meters
* Vertical Datum = NADV88
* Vertical Units = Meters
* Output Product = Point
* Output Format = Points - LAS
* Data Classes = Ground 

<u>Step 1.5:</u> Click next, add email, review and submit

*It may take a few minutes for the data to process but you will recieve an email with the link to download the data. Once it downloads, extract the data to your machine and note the file path.*

<u> Step 2: Process into a 1m DEM</u> 

In [ ]:
# Load Necessary Packages
import arcpy
import os

# Set the workspace 
#arcpy.env.workspace = "C:/path_to_lidar_data"
arcpy.env.workspace = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
arcpy.env.overwriteOutput = True

In [2]:
# Define LAS data and LAS dataset paths 
#**Can I make these user inputs?**
#las_folder = "C:/path_to_lidar_data"
#las_dataset = "C:/path_to_lidar_data/las_dataset.lasd"
las_folder = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
las_dataset = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/nc2019_dunex_Job1099351.lasd"

# Create a LAS Dataset using Create LAS Dataset Tool 
arcpy.management.CreateLasDataset(las_folder, las_dataset)

<Result 'C:\\Users\\mgm79\\Downloads\\nc2019_dunex_Job1099351\\nc2019_dunex_Job1099351.lasd'>

In [25]:
# Define output raster (DEM) path 
ground_DEM = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/DEM_1m.tif"

# Create 1m DEM using LAS Dataset to Raster tool 
arcpy.conversion.LasDatasetToRaster(las_dataset, ground_DEM, 
                                    'ELEVATION', 
                                    'BINNING AVERAGE LINEAR', 
                                    'FLOAT', 
                                    'CELLSIZE', 1, 1)


<Result 'C:\\Users\\mgm79\\Downloads\\nc2019_dunex_Job1099351\\DEM_1m.tif'>

### 2. Tidal Variability Surface Using Vertical Datum Tool

*Use DEM created above to create a tidal surface raster that shows the tidal fluctuations at the area of interest. Once this is created then sea level rise inundation can be calculated.*

In [ ]:
# Create Points from DEM to input into VDatum tool to get conversion from MHHW into NAVD88

# Raster to Point - set local variables 
in_DEM = ground_DEM
out_point_features = 'DEM_points.shp'
field = "VALUE"

# Run the tool
# arcpy.conversion.RasterToPoint(in_DEM, out_point_features, field)

#IS THERE AN EASIER WAY TO DO THIS?

### 3. Calculate Inundation Extent under Desired SLR Amount

*Use both the DEM in step 1 and the tidal variabliity surface in step 2 to model the desired amount of sea level rise at 1 meter resolution on top of the area of interest. For detailed methods see Deatiled Method for Mappnig Sea Level Rise Inundation (NOAA, 2017).*

<u> Step 3.1: Add desired SLR amount to tidal raster </u>

In [ ]:
import arcpy
from arcpy.ia import *
from arcpy.sa import *

# Allow outputs to be overwritten
arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

# Set local variables
ground_DEM = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/DEM_1m.tif"
#tidal_raster = arcpy.Raster("tidal_variability.tif")
SLR_1ft = 0.3048 
SLR_2ft = 0.6096
SLR_3ft = 0.9144 
surface_raster = '../Scratch/surface.tif'

# Execute Plus tool 
arcpy.ddd.Plus(in_raster_or_constant1=ground_DEM, 
               in_raster_or_constant2=SLR_1ft, 
               out_raster=surface_raster)


'../Scratch/surface.tif'

<u> Step 3.2: Subtract DEM values from water surface to derive initial inundatino depth grid </u>

In [ ]:
single = "../Scratch/single.tif"
Raster_Calculator_1 = single
single =  Con(ground_DEM <= surface_raster, 1)
single.save(Raster_Calculator_1)


In [ ]:
# Change the working environment to where rasters are saved
# Add user input for this? arcpy.env.workspace = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
from arcpy import sa

# Set local variables
#tidal_DEM 
#DEM_output 
SLR_1ft = 0.3048 
SLR_2ft = 0.6096
SLR_3ft = 0.9144 

#output_plus = "surface_SLR_1ft.tif" #create this as a function of what the user inputs for SLR amount
# Add desired amount of SLR to tidal variability raster  --> CAN THIS ALSO BE A USER INPUT?
## Remember this will be in *meters* not feet

# Plus tool 
outPlus = sa.Plus(ground_DEM, SLR_1ft)

# Save the output
outPlus.save('../Scratch/surface.tif')

# View the new surface 
#outPlus

#### References 

#### Dobson, G., Johnson, I., Rhodes, K., Hutchins, M., & Chesnutt, M. (2019). Regional coastal resilience assessment: Methodology. National Fish and Wildlife Foundation. Retrieved November 21, 2024, from https://www.nfwf.org/sites/default/files/coastalresilience/Documents/regional-coastal-resilience-assessment.pdf

#### National Oceanic and Atmospheric Administration. (2017, January). Sea level rise inundation methods: Technical documentation. NOAA Office for Coastal Management. Retrieved November 21, 2024, from https://coast.noaa.gov/data/digitalcoast/pdf/slr-inundation-methods.pdf